In [23]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col,  to_timestamp, monotonically_increasing_id
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#not required to load from config since this note book has the permission to the EMR Cluster and this notebook is running on AWS.
#config = configparser.ConfigParser()
#config.read_file(open('dl.cfg'))
#print(config.sections())

In [24]:
#os.environ['AWS_ACCESS_KEY_ID']=config['AWS_KEY']['AWS_ACCESS_KEY_ID']
#os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_KEY']['AWS_SECRET_ACCESS_KEY']

spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
input_data = os.path.join("s3a://udacity-dend/")
output_data = "s3a://vin-udacity-data-lake-project/"
log_data = spark.read.json(input_data+"/log_data/*/*/*.json")
log_data.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+--------------------+------+
|  artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|         song|status|           ts|           userAgent|userId|
+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+--------------------+------+
|Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
+--------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+-------------+------+-------------+----

In [26]:
song_data = spark.read.json(input_data+"/song_data/*/*/*")
song_data.show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
|AR4T2IF1187B9ADBB7|       63.96027|<a href="http://b...|        10.22442| Billy Idol|233.22077|        1|SOVIYJY12AF72A4B00|The Dead Next Doo...|1983|
+------------------+---------------+--------------------+----------------+-----------+---------+---------+------------------+--------------------+----+
only showing top 1 row

In [27]:
log_data.printSchema()
song_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable =

In [28]:
songs_data_table = song_data['song_id','title','artist_id','year','duration'].dropDuplicates(['song_id'])
songs_data_table.write.partitionBy('year','month').parquet(os.path.join(output_data,'songs.parquet'),'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
artists_data_table = song_data['artist_id','artist_name','artist_location','artist_latitude','artist_longitude'].dropDuplicates(['artist_id'])
artists_data_table.write.parquet(os.path.join(output_data,'artists.parquet'),'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#log_data.createOrReplaceTempView("staging_events")
#spark.sql("SELECT *, cast(ts/1000 as Timestamp) as timestamp from staging_events where page='NextSong'").show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+--------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|           timestamp|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+--------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:...|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| fr

In [38]:
filter = log_data['page'] == 'NextSong'
user_table = log_data['userId','firstName','lastName','gender','level'].where(filter).dropDuplicates(['userId'])
user_table.write.parquet(os.path.join(output_data, 'user.parquet'), 'overwrite')
user_table.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    51|     Maia|   Burke|     F| free|
|     7|   Adelyn|  Jordan|     F| free|
+------+---------+--------+------+-----+
only showing top 2 rows

In [39]:
get_timestamp = udf(lambda x:str(int(int(x)/1000)))
get_datetime = udf(lambda x:str(datetime.fromtimestamp(int(x)/1000.0)))
log_data = log_data.withColumn('timestamp',get_timestamp(log_data.ts))
log_data = log_data.withColumn('datetime',get_datetime(log_data.ts))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
time_table = log_data.select(
    col('datetime').alias('start_time'),
    hour('datetime').alias('hour'),
    dayofmonth('datetime').alias('day'),
    weekofyear('datetime').alias('week'),
    month('datetime').alias('month'),
    year('datetime').alias('year')                        
)
time_table.dropDuplicates(['start_time'])
time_table.show(5)
#Write to s3
time_table.write.partitionBy('year','month').parquet(os.path.join(output_data, 'time.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+---+----+-----+----+
|          start_time|hour|day|week|month|year|
+--------------------+----+---+----+-----+----+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|
|2018-11-15 01:57:...|   1| 15|  46|   11|2018|
|2018-11-15 03:29:...|   3| 15|  46|   11|2018|
+--------------------+----+---+----+-----+----+
only showing top 5 rows

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 34524)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)

In [41]:
log_data.printSchema()
song_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- datetime: string (nullable = true)

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable

In [43]:
log_data = log_data.join(song_data, song_data.title == log_data.song)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:

songplays_table = log_data.select(
    col('ts').alias('start_time'),
    col('userId').alias('user_id'),
    col('level').alias('level'),
    col('song_id').alias('song_id'),
    col('artist_id').alias('artist_id'),
    col('sessionId').alias('session_id'),
    col('location').alias('location'),
    year('datetime').alias('year'),
    month('datetime').alias('month')
)
songplays_table.select(monotonically_increasing_id().alias('songplay_id')).collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(songplay_id=8589934592), Row(songplay_id=25769803776), Row(songplay_id=34359738368), Row(songplay_id=34359738369), Row(songplay_id=68719476736), Row(songplay_id=103079215104), Row(songplay_id=103079215105), Row(songplay_id=111669149696), Row(songplay_id=111669149697), Row(songplay_id=137438953472), Row(songplay_id=137438953473), Row(songplay_id=137438953474), Row(songplay_id=137438953475), Row(songplay_id=137438953476), Row(songplay_id=137438953477), Row(songplay_id=137438953478), Row(songplay_id=137438953479), Row(songplay_id=137438953480), Row(songplay_id=137438953481), Row(songplay_id=146028888064), Row(songplay_id=163208757248), Row(songplay_id=188978561024), Row(songplay_id=188978561025), Row(songplay_id=188978561026), Row(songplay_id=188978561027), Row(songplay_id=188978561028), Row(songplay_id=188978561029), Row(songplay_id=188978561030), Row(songplay_id=197568495616), Row(songplay_id=206158430208), Row(songplay_id=206158430209), Row(songplay_id=206158430210), Row(songplay_

In [ ]:
songplays_table.write.partitionBy('year','month').parquet(os.path.join(output_data, 'songplays.parquet'), 'overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
## Clean up
spark.stop()

In [ ]:
songplays_table.printSchema()